In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from myst_nb import glue
from slugify import slugify

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc
import setvariables as conf_


import matplotlib as mpl


format_kwargs = dict(precision=2, thousands="'", decimal=",")
def make_exportable(data, file_name, cmap='YlOrBr'):
    t.fillna(0, inplace=True)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.heatmap(data=t, vmin=0, vmax=1, cmap=cmap, annot=True, fmt='.2', annot_kws={'size':10}, ax=ax, cbar=False)
    plt.tight_layout()
    ax.tick_params(which='both', axis='both', bottom=False, left=False)
    plt.savefig(file_name, dpi=300)

    plt.close()

glue('blank_caption', " ", display=False)

# Déchets Sauvages

Les déchets sauvages sur les plages du Lac Léman font l'objet d'une surveillance depuis 2015. La campagne la plus récente avant Plastock a été parrainée par l'Office fédéral de l'environnement (OFEV) de 2020 à 2021. Les directives européennes sur la surveillance des déchets sauvages sur les plages ont été publiées en 2013. Les méthodes utilisées pour surveiller les déchets sauvages sur le lac Léman suivent les directives de l'UE, tout en tenant compte des contraintes liées aux différences de topographie entre le littoral maritime et le littoral lacustre.

## Le système de mesure.

Les déchets sauvages sur les plages sont mesurés en unités de déchets par mètre linéaire de rivage. Les relevés sont effectués entre la ligne d'eau (là où l'eau et la terre se rejoignent) et la ligne de dérive (la distance la plus éloignée de la ligne d'eau atteinte par les vagues). Cette méthode a été adoptée dès le début en Suisse, ce qui signifie que les 250 échantillons précédents du lac Léman ont été collectés et mesurés selon la méthode précédemment décrite.

__Dans cette section, les unités sont des déchets par mètre de rivage: (pcs/m)__

### Exigences particulières données plastock

La quantité de déchets sauvages par mètre de plage correspond au nombre total d'objets identifiés divisé par la longueur du rivage. Pour les données Plastock, cela signifie que nous devons considérer la position un (ligne d'eau) et la position deux (plage sèche) ensemble. De plus, il y a __douze échantillons où le substrat était différent entre la position un et la position deux__. Par conséquent, ces 12 échantillons ont tous été classés dans la catégorie Sables grossiers."

In [2]:
new_data = pd.read_csv("data/end_pipe/macro_current.csv")
beach_data = pd.read_csv("data/pstock_beaches_current.csv")
codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')

new_column_names = {
    "Position":"position",
    "Substrat":"substrat",
    "Date":"date",
    "Code":"code",
    "Quantité":"quantité",
    "Aire":"area"
}

length_key = beach_data[["Plage","length"]].drop_duplicates("Plage").set_index("Plage")
work_data = new_data[["Plage", *new_column_names.keys()]].copy()
work_data.rename(columns=new_column_names, inplace=True)
work_data["length"] = work_data.Plage.apply(lambda x: length_key.loc[x, "length"])
work_data["slug"] = work_data.Plage.apply(lambda x: slugify(x))
work_data["echantillon"] = list(zip(work_data.slug, work_data['date']))
work_data['date'] = pd.to_datetime(work_data["date"], format="mixed", dayfirst=True)
work_data.dropna(inplace=True)
work_data[["position", "substrat"]] = work_data[["position", "substrat"]].astype("int")
work_data['échantillon'] = work_data['echantillon']
work_data.drop(['echantillon'], inplace=True, axis=1)
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m'] = work_data['quantité']/work_data['length']


voi = 'substrat'
vals = "pcs/m"
some_data = work_data.copy()
groupby = ['échantillon', voi]
data = some_data.groupby(groupby, as_index=False)[vals].sum()
dd = data[data['échantillon'].duplicated()].copy()

# duplicates = work_data[work_data['échantillon'].isin(ddd)].copy()
duplicated = work_data[work_data['échantillon'].isin(dd['échantillon'].unique())].copy()
duplicated['substrat'] = 2 

# notduplicated
not_duplicated = work_data[~(work_data['échantillon'].isin(dd['échantillon'].unique()))].copy()

# put it back to gether again
work_data = pd.concat([duplicated, not_duplicated])
work_data = work_data.groupby(['échantillon', 'Plage', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m'] = work_data['quantité']/work_data['length']

In [3]:
work_data.groupby('substrat')['échantillon'].nunique()/work_data['échantillon'].nunique()

substrat
1    0.275510
2    0.316327
3    0.163265
4    0.244898
Name: échantillon, dtype: float64

In [4]:
new_names = {x: " " for x in work_data.columns}
new_names = {
    'échantillon': 'loc_date',
    'pcs/m': 'pcs_m'
}

change_names = ['preverenges', 'tolochenaz', 'versoix', 'vidy', 'cully']

changeus = work_data[work_data.slug.isin(change_names)].copy()
donotchange = work_data[~work_data.slug.isin(change_names)].copy()

new_slug = {
    'cully': 'cully-p',
    'preverenges': 'preverenges-p',
    'tolochenaz': 'tolochenaz-p',
    'versoix':'versoix-p',
    'vidy': 'vidy-p'}

# they have the same name as locations in iqaasl
changeus['new_slug'] = changeus.slug.apply(lambda x: new_slug[x])
changeus['slug'] = changeus.new_slug
changeus.drop('new_slug', inplace=True, axis=1)
wd_nn = pd.concat([changeus, donotchange])

# codes in the plastock data
pcodes = wd_nn.code.unique()

# identify and remove codes for which there is no defintion
t = [x for x in pcodes if x not in codes.index]
wd_ni = wd_nn[~wd_nn.code.isin(t)].copy()

# aggregate to grfags, gfoams, gcaps
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)

# aggregate along all land-use and topo variables.
ti = ti.groupby(['échantillon', 'Plage', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})

# make city map and format plastock data to work with report class
f = pd.read_csv('data/u_pstk.csv')
city_map = f[["slug", "city"]].drop_duplicates()
city_map.loc[city_map.slug == 'savoniere', 'slug'] = 'savonniere'
city_map.set_index('slug', inplace=True)

# adding and renaming columns according to reportclass requirements
ti['city'] = ti.slug.apply(lambda x: city_map.loc[x])
ti['feature_name'] = 'lac-leman'
ti['feature_type'] = 'l'
ti['parent_boundary'] = 'rhone'
ti.rename(columns={**new_names,'quantité': 'quantity'}, inplace=True)
ti['pcs_m'] = ti.quantity/ti.length


twork_cols = ['loc_date', 'date', 'slug',
       'code', 'quantity', 'city', 'feature_name', 'feature_type',
       'parent_boundary', 'pcs_m']

ti_work = ti[twork_cols].copy()

# previous results
ghi = pd.read_csv('data/end_pipe/iqaasl.csv')

ti_work['project']='Plastock'
# combine zones and substrat
comb_cols = ['loc_date', 'date', 'slug',
        'city','feature_name', 'feature_type',
       'parent_boundary','code']

ti_work = ti_work.groupby(comb_cols, as_index=False).agg(conf_.unit_agg)
ti_work['project']='Plastock'
txi = pd.concat([ghi, ti_work])
txi.reset_index(inplace=True)
txi = txi[txi.code.isin(ti_work.code.unique())]

language_maps = rc.language_maps()
boundaries = dict(start_date="2021-12-31", end_date="2023-01-01", feature_name="lac-leman", language="fr")
top_label= ['feature_name', 'lac-leman']

tiii = rc.ReportClass(ti_work.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)
header = tiii.a_short_description.fillna('.')
comp_summary = tiii.summarize_feature_labels(feature='feature_name')
most_common, weight = tiii.most_common

boundaries = dict(start_date="2019-12-31", end_date="2023-01-01", feature_name="lac-leman", language="fr")
combined = rc.ReportClass(txi.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)

## Situation

Les déchets sauvages ont été collectés dans 25 emplacements différents autour du Léman. Soixante-sept sur 98 de ces emplacements se trouvaient dans des zones urbaines, avec un taux d'utilisation élevé de 55 sur 98, et 83 sur 98 étaient situés à moins de 500 mètres d'un parking. Les caractéristiques ont été notées par les personnes chargées de la collecte des déchets sauvages [Macro déchets plage et attribut](macro-attributes).



### La répartition des résultats

La médiane du total des échantillons était de 2 pièces/mètre avec une variance de 3,6 pièces/mètre.

In [23]:
t = rc.translated_and_style_for_display(comp_summary.copy(),  tiii.lang_maps[tiii.language], tiii.language, gradient=False)
rep_res_one = "L'échantillonnage était planifié trimestriellement, commençant en janvier 2022. À quelques exceptions près, chaque emplacement était échantillonné quatre fois au cours de la période de 12 mois. Les études précédentes sur le lac étaient mensuelles ou aléatoires."
rep_res_two = "La médiane du total des échantillons était de 2 pièces/mètre avec une variance de 3,6 pièces/mètre."

rep_text = f"""
{rep_res_one}<br/>

{rep_res_two}
""" 

glue('rep_results_text',rep_text , display=False)
glue('rep_results', t, display=False)

<style>
table th:first-of-type {
    width: 50%;
}
table th:nth-of-type(2) {
    width: 50%;
    padding:10px;
}

</style>


| |La répartition des résultats|
|:---:|:---|
|{glue}`rep_results`|{glue:text}`rep_results_text`|

In [6]:
comb_dt = tiii.w_df
comb_dt = comb_dt.groupby(['loc_date', 'date', 'project'], as_index=False).pcs_m.sum()

import matplotlib.dates as mdates
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

fig, ax = plt.subplots(2,2, figsize=(8,8))

sns.scatterplot(data=comb_dt, x="date", y="pcs_m", hue='project', ax=ax[0,0])
#ax[0,0].tick_params(labelbottom=False, bottom=False)
ax[0,0].set_title("Total par échantillon", loc="left")
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%y'))
ax[0,0].set_ylabel("")
capitalize_x_and_y_axis_labels(ax[0,0])
capitalize_legend_components(ax[0, 0])

sns.boxplot(data=comb_dt, y="pcs_m", x='project', hue='project', dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_title("Boîte de Tukey", loc="left")
capitalize_x_and_y_axis_labels(ax[0,1])
ax[0, 1].get_legend().remove()
ax[0,1].set_xlabel("")

sns.histplot(data=comb_dt, x="pcs_m", ax=ax[1,0], stat="probability", hue='project', kde=True)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_title("Histogramme", loc="left")
capitalize_x_and_y_axis_labels(ax[1,0])
ax[1, 0].get_legend().remove()

sns.ecdfplot(data=comb_dt, x="pcs_m", hue='project', ax=ax[1,1])
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
sns.move_legend(ax[1, 1], title=" ", loc='best')

plt.subplots_adjust(wspace=.3, hspace=.3)

glue('fig-D1', fig, display=False)
plt.close()


```{glue:figure} fig-D1
---
name: fig-D1
---
{glue:text}`blank_caption` 
```

## Les objets les plus courants Plastock

### Définition des 'objets les plus courants'

Les objets les plus courants peuvent être sélectionnés de plusieurs manières. On peut également les appeler les 'objets d'intérêt'. Dans le cadre de ce rapport, nous nous concentrons sur les objets qui représentent une proportion plus importante des résultats que les autres. Nous avons utilisé deux critères de sélection : i. la quantité, ii. le taux d'échec.

1. Si un objet a une quantité totale qui le place dans les dix premiers, il est considéré comme 'courant'.
2. Si un objet a été trouvé dans au moins la moitié des échantillons, il est ÉGALEMENT considéré comme 'courant'.

Par conséquent, pour cette étude, les 'objets les plus courants' sont ceux qui se trouvent soit dans les dix premiers en termes de nombre total de pièces de déchets ET/OU qui ont été trouvés dans au moins 50% des enquêtes. Pour Plastock, les objets les plus courants représentent 89% du montant total ou 24'156/27'493.

```{note}
Les équipes peuvent avoir d'autres critères pour s'intéresser à l'occurrence d'un objet dans l'environnement. Il s'agit d'un sujet qui devrait être largement développé au sein des équipes. 
```

In [7]:
rc.translated_and_style_for_display(most_common.copy(),  tiii.lang_maps[tiii.language], tiii.language, gradient=False)

,Quantité,% du total,Pcs/m,Taux d'échec
"Fragments de plastique: g80, g79, g78, g77, g76, g75",11'221,"0,41","0,81","0,97"
Mégots et filtres à cigarettes,3'089,"0,11","0,27","0,79"
"Emballages de bonbons, de snacks",2'080,"0,08","0,22","0,74"
Fragments de plastique angulaires <5mm,1'926,"0,07","0,27","0,41"
Pellets industriels (gpi),1'526,"0,06","0,17","0,36"
"Fragments de polystyrène expansé: g81, g82, g83",1'399,"0,05","0,14","0,72"
"Couvercles en plastique bouteille: g21, g22, g23, g24",1'070,"0,04","0,10","0,65"
Coton-tige,1'040,"0,04","0,19","0,54"
Mousse de plastique pour l'isolation thermique,406,"0,01","0,07","0,38"
Déchets de construction en plastique,380,"0,01","0,04","0,24"


### Correspondance avec les résultats d'OSPAR

La Suisse et la France sont toutes deux parties contractantes de la convention [OSPAR](https://www.ospar.org/organisation/contracting-parties). Parmi les objets les plus courants identifiés à partir des données Plastock, six peuvent également être retrouvés dans le rapport d'état de qualité le plus récent d'OSPAR concernant l'Abondance, la Composition et les Tendances des [déchets sauvages](https://www.ospar.org/work-areas/eiha/marine-litter/assessment-of-marine-litter/beach-litter).

1. Fragments de plastique : g80, g79, g78, g77, g76, g75
2. Fragments de polystyrène expansé : g81, g82, g83
3. Couvercles en plastique de bouteille : g21, g22, g23, g24
4. Coton-tige
5. Emballages de bonbons, de snacks
6. Déchets de construction en plastique

### Correspondance avec l'IQAASL

Les objets les plus courants répertoriés pour le lac Léman en 2021 comprennent tous les objets les plus courants de Plastock à l'exception des 'Fragments de plastique angulaires <5 mm', [Lac Léman, IQAASL](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/).

### Les plus courants par ville

In [8]:
w_df = tiii.w_df.copy()
cities = w_df.city.unique()
cone = cities[:12]
ctwo = cities[12:]

t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=cone)
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Amphion,Anthy,Aubonne,Bouveret,Clarens,Crans,Cully,Excenevex,Genève,Gland,Grangettes,Hermance,Cumulé
Fragments de plastique angulaires <5mm,"0,19",nan,"0,03","0,68","0,02",nan,"0,06","1,61","0,46",nan,"0,69",nan,"0,27"
Pellets industriels (gpi),"0,01","0,01",nan,"0,42","0,05",nan,nan,"1,20","0,27","0,22","0,27",nan,"0,17"
Mégots et filtres à cigarettes,"0,13","0,08","0,19","0,43","1,19",nan,"0,36","0,36","0,36","0,13","0,34","0,17","0,27"
"Emballages de bonbons, de snacks","0,43","0,01","0,08","0,43","0,24","0,07",nan,"0,14","0,16",nan,"1,94","0,06","0,22"
Bâtonnets de sucette,"0,12","0,02","0,02","0,15","0,08","0,05",nan,"0,06",nan,nan,"0,05","0,09","0,06"
Mousse de plastique pour l'isolation thermique,"0,05",nan,"0,06","0,02","0,31",nan,nan,"0,03","0,02","0,26","0,68","0,03","0,07"
Déchets de construction en plastique,"0,04","0,02",nan,"0,49","0,05",nan,nan,nan,"0,03","0,04","0,33",nan,"0,04"
Coton-tige,"0,70","0,04",nan,"0,24","0,27",nan,nan,"0,33","0,03","0,09","0,55",nan,"0,19"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,65",nan,"0,07","0,41","0,05",nan,"0,14","0,06","0,06","0,04","0,33","0,02","0,10"
"Fragments de polystyrène expansé: g81, g82, g83","0,45","0,04","0,03","0,30","0,21",nan,nan,"0,09","0,05","0,48","1,31","0,02","0,14"


In [9]:
file_name = 'resources/images/most_common_one.jpg'
make_exportable(t, file_name)

In [10]:
t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=ctwo)
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Lugrin,Lutry,Meillerie,Préverenges,Rolle,Saint-disdille,Savonière,Tolochenaz,Tougues,Versoix,Vevey,Vidy,Cumulé
Fragments de plastique angulaires <5mm,"0,01","0,88","0,10","0,49","0,61","0,07","0,27","0,08","0,04","0,92",nan,"0,18","0,27"
Pellets industriels (gpi),nan,nan,"0,04","0,82","0,11","0,01",nan,nan,nan,"0,26",nan,"0,15","0,17"
Mégots et filtres à cigarettes,"0,09","0,32","0,12","0,42","0,66","0,34","0,19","0,06","0,54","0,58","0,05","1,58","0,27"
"Emballages de bonbons, de snacks","0,31","0,10","0,24","0,26","0,23","0,62","0,29","0,14","0,13","0,20","0,07","0,65","0,22"
Bâtonnets de sucette,"0,03","0,25",nan,"0,02","0,04","0,08","0,07","0,03","0,02","0,10",nan,"0,06","0,06"
Mousse de plastique pour l'isolation thermique,nan,nan,"0,04",nan,"0,10","0,07",nan,nan,"0,05","0,13",nan,"0,06","0,07"
Déchets de construction en plastique,"0,05",nan,nan,"0,03","0,05","0,02","0,02",nan,"0,01","0,05",nan,nan,"0,04"
Coton-tige,"0,14",nan,"0,06","0,09","0,15","0,23","0,15",nan,"0,01","0,20",nan,"0,30","0,19"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,07","0,04",nan,"0,03","0,16","0,31","0,14",nan,nan,"0,13","0,06","0,16","0,10"
"Fragments de polystyrène expansé: g81, g82, g83","0,46","0,09","0,36","0,18","0,16","0,15","0,02","0,07","0,07","0,05",nan,"0,01","0,14"


In [11]:
file_name = 'resources/images/most_common_two.jpg'
make_exportable(t, file_name)

### Les plust courants et IQAASL

In [12]:
t = rc.a_cumulative_report(combined.w_df[combined.w_df.code.isin(most_common.index)], feature_name='project', object_column='code')
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Iqaasl,Plastock,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,27","0,00"
Pellets industriels (gpi),"0,00","0,17","0,03"
Mégots et filtres à cigarettes,"0,47","0,27","0,35"
"Emballages de bonbons, de snacks","0,22","0,22","0,22"
Bâtonnets de sucette,"0,03","0,06","0,04"
Mousse de plastique pour l'isolation thermique,"0,10","0,07","0,09"
Déchets de construction en plastique,"0,06","0,04","0,05"
Coton-tige,"0,13","0,19","0,16"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,11","0,10","0,11"
"Fragments de polystyrène expansé: g81, g82, g83","0,24","0,14","0,17"


In [13]:
file_name = 'resources/images/most_common_iqaasl.jpg'
make_exportable(t, file_name)

## Plastock - IQAASL

Il s'agit uniquement des objets identifiés dans le projet Plastock.

### La répartition des totaux des échantillons

In [14]:

t = combined.summarize_feature_labels(feature='project')
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=False)

In [15]:
comb_dt = combined.w_df
comb_dt = comb_dt.groupby(['loc_date', 'date', 'project'], as_index=False).pcs_m.sum()

import matplotlib.dates as mdates
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

fig, ax = plt.subplots(2,2, figsize=(8,8))

sns.scatterplot(data=comb_dt, x="date", y="pcs_m", hue='project', ax=ax[0,0])
#ax[0,0].tick_params(labelbottom=False, bottom=False)
ax[0,0].set_title("Total par échantillon", loc="left")
ax[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax[0,0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%y'))
ax[0,0].set_ylabel("")
capitalize_x_and_y_axis_labels(ax[0,0])
capitalize_legend_components(ax[0, 0])

sns.boxplot(data=comb_dt, y="pcs_m", x='project', hue='project', dodge=False, width=.9, ax=ax[0,1])
ax[0,1].set_title("Boîte de Tukey", loc="left")
capitalize_x_and_y_axis_labels(ax[0,1])
ax[0, 1].get_legend().remove()
ax[0,1].set_xlabel("")

sns.histplot(data=comb_dt, x="pcs_m", ax=ax[1,0], stat="probability", hue='project', kde=True)
ax[1,0].yaxis.set_major_formatter('{x:.2f}')
ax[1,0].set_ylabel("Probabilité")
ax[1,0].set_title("Histogramme", loc="left")
capitalize_x_and_y_axis_labels(ax[1,0])
ax[1, 0].get_legend().remove()

sns.ecdfplot(data=comb_dt, x="pcs_m", hue='project', ax=ax[1,1])
ax[1,1].set_title("Fonction de répartition", loc="left")
capitalize_x_and_y_axis_labels(ax[1,1])
sns.move_legend(ax[1, 1], title=" ", loc='best')

plt.subplots_adjust(wspace=.3, hspace=.3)

glue('fig-D2', fig, display=False)
plt.close()


```{glue:figure} fig-D2
---
name: fig-D2
---
{glue:text}`blank_caption` 
```

#### Les objets les plus courants Plastock + Iqaasl

In [16]:
com_mc, weight = combined.most_common
rc.translated_and_style_for_display(com_mc.copy(), tiii.lang_maps[tiii.language], tiii.language, gradient=False)

,Quantité,% du total,Pcs/m,Taux d'échec
"Fragments de plastique: g80, g79, g78, g77, g76, g75",15'996,"0,30","0,69","0,97"
Mégots et filtres à cigarettes,6'255,"0,12","0,35","0,87"
"Fragments de polystyrène expansé: g81, g82, g83",5'040,"0,10","0,17","0,78"
"Emballages de bonbons, de snacks",3'917,"0,07","0,22","0,85"
Pellets industriels (gpi),2'966,"0,06","0,03","0,42"
Fragments de plastique angulaires <5mm,2'357,"0,04","0,00","0,33"
"Couvercles en plastique bouteille: g21, g22, g23, g24",2'345,"0,04","0,11","0,77"
Coton-tige,2'234,"0,04","0,16","0,68"
Mousse de plastique pour l'isolation thermique,1'531,"0,03","0,09","0,58"
"Bâche, feuille plastique industrielle",1'279,"0,02","0,10","0,43"


In [17]:
weight

{'quantity': 46003.0, '%': 0.871648634821987}

#### Distribution des résultats pour chaque projet.

In [18]:
t = rc.a_cumulative_report(combined.w_df[combined.w_df.code.isin(com_mc.index)], feature_name='project', object_column='code')
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Iqaasl,Plastock,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,27","0,00"
Pellets industriels (gpi),"0,00","0,17","0,03"
Polystyrène < 5mm,"0,00","0,07","0,00"
Mégots et filtres à cigarettes,"0,47","0,27","0,35"
"Emballages de bonbons, de snacks","0,22","0,22","0,22"
Bâtonnets de sucette,"0,03","0,06","0,04"
Pailles et agitateurs,"0,04","0,03","0,04"
"Bâche, feuille plastique industrielle","0,09","0,12","0,10"
Mousse de plastique pour l'isolation thermique,"0,10","0,07","0,09"
Coton-tige,"0,13","0,19","0,16"


In [19]:
file_name = 'resources/images/most_common_combined.jpg'
make_exportable(t, file_name)

In [20]:
## Perspectives 2024.

## Inventaire Plastock.

In [21]:
t = rc.translate_for_display(tiii.inventory, amap=language_maps['fr'], lan='fr')
t['objet'] = t.code.apply(lambda x: codes.loc[x, 'fr'])
t = t[[t.columns[0], t.columns[-1], *t.columns[1:-1]]]
t.set_index(['code', 'objet'], inplace=True)
t.index.name = None
t.style.set_table_styles(conf_.table_css_styles).format(**conf_.format_kwargs)

,,quantité,% du total,pcs/m,taux d'échec
code,objet,,,,
Gfrags,"Fragments de plastique: G80, G79, G78, G75",11'221,"0,41","0,81","0,97"
G27,Mégots et filtres à cigarettes,3'089,"0,11","0,27","0,79"
G30,"Emballages de bonbons, de snacks",2'080,"0,08","0,22","0,74"
G106,Fragments de plastique angulaires <5mm,1'926,"0,07","0,27","0,41"
G112,Pellets industriels (GPI),1'526,"0,06","0,17","0,36"
Gfoams,"Fragments de polystyrène expansé: G76, G81, G82, G83",1'399,"0,05","0,14","0,72"
Gcaps,"Couvercles en plastique bouteille: G21, G22, G23, G24",1'070,"0,04","0,10","0,65"
G95,Coton-tige,1'040,"0,04","0,19","0,54"
G74,Mousse de plastique pour l'isolation thermique,406,"0,01","0,07","0,38"


In [22]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: nov29

matplotlib: 3.7.1
seaborn   : 0.12.2
pandas    : 2.0.0
numpy     : 1.24.2

